In [1]:
from core import isb, database
import pandas as pd
import datetime

from isb.finance import Bond, BondType, daycount as dc, utils

In [183]:



def calculate_cost(prepayment: int=1, fix_period: int=5, rate_discount: float=2e-2, inflation=3.5e-2):

    penalty_charge = 0.2e-2*(fix_period-prepayment-1)
    penalty_charge = max([penalty_charge, 0])

    original_principal = 50e6
    original_interest_rate = 4.00e-2
    original_duration = 25
    
    issue_date=datetime.date(2025,12,1)
    prepayment_date = datetime.date(2025+prepayment,12,1)
    maturity_date=datetime.date(2025+original_duration,12,1)
    
    original = Bond(
        "",
        original_principal,
        bond_type=BondType.annuity,
        coupon_rate=original_interest_rate,
        coupon_frequency=12,
        day_count_convension=dc.thirtyE_360,
        issue_date=issue_date,
        maturity_date=maturity_date,
        cost_per_payment=130,
        settings__simple_coupon_rate=True,
        indexation=inflation
    )

    try:
        principal_remaining = \
            original.get_cashflow(reference_date=issue_date)\
            .set_index('date')\
            .loc[prepayment_date]\
            .principal_remaining
    except KeyError:
        principal_remaining = original_principal

    bond1 = Bond(
        "",
        principal_remaining,
        bond_type=BondType.annuity,
        coupon_rate=original_interest_rate,
        coupon_frequency=12,
        day_count_convension=dc.thirtyE_360,
        issue_date=prepayment_date,
        maturity_date=maturity_date,
        cost_per_payment=130,
        prepayment_time=fix_period-prepayment,
        settings__simple_coupon_rate=True,
        indexation=inflation
    )
    bond2 = Bond(
        "",
        principal_remaining,
        bond_type=BondType.annuity,
        coupon_rate=original_interest_rate - rate_discount,
        coupon_frequency=12,
        day_count_convension=dc.thirtyE_360,
        issue_date=prepayment_date,
        maturity_date=maturity_date,
        cost_per_payment=130,
        prepayment_time=fix_period-prepayment,
        settings__simple_coupon_rate=True,
        indexation=inflation
    )

    discount_rate = utils.calculate_coupon_rate(original_interest_rate, 12)

    PV_CP_1 = bond1.get_cashflow(reference_date=prepayment_date)\
        .apply(lambda x: utils.discount(x.total_payment, discount_rate, x.time_to_payment), axis=1)\
        .sum()
    PV_CP_2 = bond2.get_cashflow(reference_date=prepayment_date)\
        .apply(lambda x: utils.discount(x.total_payment, discount_rate, x.time_to_payment), axis=1)\
        .sum()

    return -1 * (
        PV_CP_1 - 
        PV_CP_2 - 
        0 * principal_remaining * (penalty_charge)
    ) / principal_remaining

In [186]:
pd.DataFrame([dict(i=i, cost=calculate_cost(i, 20, rate_discount=1.0e-2, inflation=0)) for i in range(0, 20)])\
.set_index('i')\
.isb.plot()

In [178]:
bond1.get_cashflow()

,payment_number,date,time_to_payment,time_from_issue,time_to_maturity,coupon_payment,principal_payment,indexation_payment,payment_cost,total_payment,principal_remaining
0,1,2027-01-01,1.113889,0.083333,23.916667,168883,105057,0,130,274070,50559723
1,2,2027-02-01,1.197222,0.166667,23.833333,169016,50704875,0,130,50874021,0


In [29]:
calculate_cost(4)

np.float64(214582.92633007467)

In [217]:
import numpy as np

# Inputs
principal = 100e6   # remaining balance after 1 year
years = 20
rate_old = 0.04          # original coupon
rate_new = 0.03          # new coupon
discount_rate = 0.04     # bank's required yield
frequency = 12           # payments per year (12 for monthly, 1 for annual)
prepayment_cost = principal * (years-1)*0.2e-2

# Per-period rates and number of periods
r_old = rate_old / frequency
r_new = rate_new / frequency
r_disc = discount_rate / frequency
n_periods = years * frequency

# Compute annuity payments for old and new rates
payment_old = principal * r_old / (1 - (1 + r_old) ** (-n_periods))
payment_new = principal * r_new / (1 - (1 + r_new) ** (-n_periods))

# Discount each payment back at the bank's required yield
pv_old = sum(payment_old / ((1 + r_disc) ** t) for t in range(1, n_periods + 1))
pv_new = sum(payment_new / ((1 + r_disc) ** t) for t in range(1, n_periods + 1))

loss = pv_old - pv_new
loss_pct = loss / pv_old * 100

print(f"PV at 4%: {pv_old:,.2f}")
print(f"PV at 3% discounted at 4%: {pv_new:,.2f}")
print(f"Loss: {loss:,.2f} ({loss_pct:.2f}%)")

loss = loss - prepayment_cost
loss_pct = loss / pv_old * 100
print(f"Absolute loss: {loss:,.2f} ({loss_pct:.2f}%)")

PV at 4%: 100,000,000.00
PV at 3% discounted at 4%: 91,520,726.18
Loss: 8,479,273.82 (8.48%)
Absolute loss: 4,679,273.82 (4.68%)


In [305]:
def calculate_banks_loss(
    period: int=1,
    interest_rate: float=0.04,
    market_rate: float=0.03,
    original_fix_period: int=20,
    mortgage_duration: int=25,
    print_results: bool=True,
) -> float:
    
    prepayment_time = (original_fix_period-period)*12

    if prepayment_time == 0:
        return 0

    m1 = Bond(
        "",
        100e6,
        bond_type=BondType.amortizing,
        coupon_rate=interest_rate,
        coupon_frequency=12,
        day_count_convension=dc.thirtyE_360,
        issue_date=datetime.date(2025,12,1),
        maturity_date=datetime.date(2025+mortgage_duration-period,12,1),
        cost_per_payment=130,
        prepayment_time=prepayment_time,
        settings__simple_coupon_rate=True
    )

    m2 = Bond(
        "",
        100e6,
        bond_type=BondType.annuity,
        coupon_rate=market_rate,
        coupon_frequency=12,
        day_count_convension=dc.thirtyE_360,
        issue_date=datetime.date(2025,12,1),
        maturity_date=datetime.date(2025+mortgage_duration-period,12,1),
        cost_per_payment=130,
        prepayment_time=prepayment_time,
        settings__simple_coupon_rate=True
    )

    penalty_cost = m1.face_value * (original_fix_period-period-1)*0.2e-2
    penalty_cost = max([penalty_cost, 0])

    pv_m1 = m1.dirty_price(interest_rate)
    pv_m2 = m2.dirty_price(interest_rate)
    loss = (pv_m2 + penalty_cost - pv_m1)/pv_m1

    if print_results:
        print(f"PV @ {interest_rate:.2%}: {pv_m1:,.0f}")
        print(f"PV @ {market_rate:.2%}, discounted @ {interest_rate:.2%}: {pv_m2:,.0f}")
        print(f"Penalty cost: {penalty_cost:,.0f}")
        print(f"Banks loss: {loss:.2%}")

    return loss

In [306]:
calculate_banks_loss(
    period=1,
    interest_rate=0.0485,
    market_rate=0.0480,
    original_fix_period=20,
    mortgage_duration=20,
    print_results=True,
)

PV @ 4.85%: 100,627,278
PV @ 4.80%, discounted @ 4.85%: 100,317,107
Penalty cost: 3,600,000
Banks loss: 3.27%


np.float64(0.03269321798147925)

In [307]:
df = pd.DataFrame([dict(
    period = i,
    market_rate=0.0485 - j/1000,
    loss = calculate_banks_loss(
        period=i,
        interest_rate=0.0485,
        market_rate=0.0485 - j/1000,
        original_fix_period=20,
        mortgage_duration=20,
        print_results=False,
    ) 
) 
    for i in range(21)
    for j in range(21)
])

In [310]:
df\
.pivot(index='market_rate', columns='period', values='loss')

period,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
market_rate,,,,,,,,,,,,,,,,,,,,,
0.0285,-0.121823,-0.117518,-0.113063,-0.108454,-0.103689,-0.098764,-0.093674,-0.088416,-0.082986,-0.077380,...,-0.065623,-0.059464,-0.053111,-0.046561,-0.039808,-0.032848,-0.025675,-0.018286,-0.010674,0.0
0.0295,-0.114176,-0.110160,-0.106003,-0.101700,-0.097248,-0.092644,-0.087885,-0.082968,-0.077887,-0.072640,...,-0.061633,-0.055864,-0.049913,-0.043776,-0.037448,-0.030925,-0.024203,-0.017277,-0.010141,0.0
0.0305,-0.106488,-0.102765,-0.098908,-0.094913,-0.090778,-0.086499,-0.082074,-0.077499,-0.072770,-0.067885,...,-0.057631,-0.052255,-0.046708,-0.040986,-0.035085,-0.029001,-0.022729,-0.016267,-0.009609,0.0
0.0315,-0.098759,-0.095332,-0.091778,-0.088094,-0.084278,-0.080328,-0.076239,-0.072009,-0.067636,-0.063115,...,-0.053620,-0.048638,-0.043496,-0.038191,-0.032717,-0.027073,-0.021254,-0.015257,-0.009076,0.0
0.0325,-0.090990,-0.087861,-0.084613,-0.081244,-0.077751,-0.074131,-0.070381,-0.066500,-0.062483,-0.058330,...,-0.049597,-0.045012,-0.040278,-0.035390,-0.030347,-0.025144,-0.019778,-0.014246,-0.008543,0.0
0.0335,-0.083180,-0.080352,-0.077414,-0.074362,-0.071194,-0.067907,-0.064500,-0.060970,-0.057313,-0.053529,...,-0.045563,-0.041377,-0.037052,-0.032585,-0.027972,-0.023212,-0.018300,-0.013234,-0.008010,0.0
0.0345,-0.075329,-0.072806,-0.070180,-0.067448,-0.064608,-0.061658,-0.058596,-0.055419,-0.052125,-0.048712,...,-0.041519,-0.037734,-0.033820,-0.029774,-0.025594,-0.021277,-0.016821,-0.012222,-0.007477,0.0
0.0355,-0.067437,-0.065223,-0.062912,-0.060503,-0.057994,-0.055384,-0.052669,-0.049848,-0.046919,-0.043881,...,-0.037464,-0.034082,-0.030581,-0.026958,-0.023212,-0.019340,-0.015340,-0.011209,-0.006944,0.0
0.0365,-0.059505,-0.057602,-0.055609,-0.053527,-0.051352,-0.049083,-0.046719,-0.044257,-0.041696,-0.039034,...,-0.033398,-0.030421,-0.027335,-0.024137,-0.020827,-0.017401,-0.013858,-0.010195,-0.006410,0.0
